In [1]:
import numpy as np
import pandas as pd
import os
import sys
import datetime

## 1 读取预报文件

In [2]:
targetpath=r"D:\git仓库\GridForecastSys\extra_data\grid_data"
filename="Sample_new.txt"
fullname=os.path.join(targetpath,filename)
print(fullname)
#data=np.loadtxt(fullname)
# data=pd.read_table(targetpath,encoding = 'utf-8',header=None)
with open(fullname,"rb") as f:
#     columns_date=pd.read_table(f,sep='\t',encoding='utf-8',nrows=1,header=None)
    data=pd.read_table(f,sep='\t',encoding='utf-8',header=None,infer_datetime_format=True)
    print('读取成功')

D:\git仓库\GridForecastSys\extra_data\grid_data\Sample_new.txt
读取成功


#### 读取第一行
 注意此时的第一行数据为一个series

In [4]:
columns_date=data.iloc[0]
columns_date.head()

0            NaN
1            NaN
2    2.01802e+09
3    2.01802e+09
4    2.01802e+09
Name: 0, dtype: object

### 数据清理
#### 处理nan
 对于series仍可以使用.notnull()与.isnull()过滤nan  
 返回的类型均为series

* 此处可以使用

#### 1-2 df.dropna(axis=1)：删除所有包含空值的列  
 注意对于Series类型的数据不能使用.dropna（）方法

In [5]:
# 转成list
columns_date_notnan=columns_date[columns_date.notnull()]
columns_date_notnan.head()
# columns_date_convert

2    2.01802e+09
3    2.01802e+09
4    2.01802e+09
5    2.01802e+09
6    2.01802e+09
Name: 0, dtype: object

#### series的values是一个numpy.ndarray

In [6]:
print(columns_date_notnan.values[0])
print(type(columns_date_notnan.values[0]))

2018022608.0
<class 'numpy.float64'>


调用map方法  
 还需去掉.0
 #### 最终采取此种方式：
 * 将第一行使用map转成int类型（去掉了.0）  
 * 再转换为str类型

In [7]:
columns_date_notnan_convert=columns_date_notnan.map(int).map(str)
print(columns_date_notnan_convert.values[0])
print(type(columns_date_notnan_convert.values[0]))

2018022608
<class 'str'>


## 3 读取除去第一行的其余数据

In [8]:
data=data[1:]
data.head()

,0,1,2,3,4,5,6,7,8,9,...,65,66,67,68,69,70,71,72,73,74
1,N01,HS,0.49,0.48,0.46,0.45,0.43,0.42,0.41,0.40,...,0.35,0.35,0.35,0.35,0.35,0.35,0.36,0.36,0.39,0.42
2,N01,PER,3.14,3.14,3.20,3.27,3.34,3.38,3.41,3.44,...,3.70,3.74,3.81,3.90,3.95,3.93,3.85,3.66,3.30,3.05
3,N01,DIR,11.19,11.21,12.50,14.01,15.01,15.63,16.05,16.31,...,17.03,17.33,17.59,17.84,17.62,16.95,15.89,13.79,9.30,5.26
4,N02,HS,0.44,0.43,0.42,0.41,0.40,0.40,0.39,0.38,...,0.31,0.32,0.32,0.32,0.32,0.32,0.32,0.34,0.39,0.45
5,N02,PER,4.03,4.05,4.06,4.04,4.02,3.99,3.94,3.94,...,4.29,4.36,4.42,4.47,4.44,4.35,4.15,3.76,3.28,3.07


## 4 更改索引

#### 此种方法才是更改索引

In [9]:
data=data.set_index([0,1])

In [10]:
data.head()

2      3      4      5      6      7      8      9      10     11  \
0   1                                                                           
N01 HS    0.49   0.48   0.46   0.45   0.43   0.42   0.41   0.40   0.39   0.38   
    PER   3.14   3.14   3.20   3.27   3.34   3.38   3.41   3.44   3.47   3.49   
    DIR  11.19  11.21  12.50  14.01  15.01  15.63  16.05  16.31  16.51  16.71   
N02 HS    0.44   0.43   0.42   0.41   0.40   0.40   0.39   0.38   0.37   0.36   
    PER   4.03   4.05   4.06   4.04   4.02   3.99   3.94   3.94   3.97   3.98   

         ...      65     66     67     68     69     70     71     72    73  \
0   1    ...                                                                  
N01 HS   ...    0.35   0.35   0.35   0.35   0.35   0.35   0.36   0.36  0.39   
    PER  ...    3.70   3.74   3.81   3.90   3.95   3.93   3.85   3.66  3.30   
    DIR  ...   17.03  17.33  17.59  17.84  17.62  16.95  15.89  13.79  9.30   
N02 HS   ...    0.31   0.32   0.32   0.32   0.32   0.32   0.32   0.34  0.39   
    PER  ...    4.29   4.36   4.42   4.47   4.44   4.35   4.15   3.76  3.28   

           74  
0   1          
N01 HS   0.42  
    PER  3.05  
    DIR  5.26  
N02 HS   0.45  
    PER  3.07  

[5 rows x 73 columns]

## 5 修改head

In [11]:
data.columns=columns_date_notnan_convert

In [12]:
data.head()

0        2018022608  2018022609  2018022610  2018022611  2018022612  \
0   1                                                                 
N01 HS         0.49        0.48        0.46        0.45        0.43   
    PER        3.14        3.14        3.20        3.27        3.34   
    DIR       11.19       11.21       12.50       14.01       15.01   
N02 HS         0.44        0.43        0.42        0.41        0.40   
    PER        4.03        4.05        4.06        4.04        4.02   

0        2018022613  2018022614  2018022615  2018022616  2018022617  \
0   1                                                                 
N01 HS         0.42        0.41        0.40        0.39        0.38   
    PER        3.38        3.41        3.44        3.47        3.49   
    DIR       15.63       16.05       16.31       16.51       16.71   
N02 HS         0.40        0.39        0.38        0.37        0.36   
    PER        3.99        3.94        3.94        3.97        3.98   

0           ...      2018022823  2018030100  2018030101  2018030102  \
0   1       ...                                                       
N01 HS      ...            0.35        0.35        0.35        0.35   
    PER     ...            3.70        3.74        3.81        3.90   
    DIR     ...           17.03       17.33       17.59       17.84   
N02 HS      ...            0.31        0.32        0.32        0.32   
    PER     ...            4.29        4.36        4.42        4.47   

0        2018030103  2018030104  2018030105  2018030106  2018030107  \
0   1                                                                 
N01 HS         0.35        0.35        0.36        0.36        0.39   
    PER        3.95        3.93        3.85        3.66        3.30   
    DIR       17.62       16.95       15.89       13.79        9.30   
N02 HS         0.32        0.32        0.32        0.34        0.39   
    PER        4.44        4.35        4.15        3.76        3.28   

0        2018030108  
0   1                
N01 HS         0.42  
    PER        3.05  
    DIR        5.26  
N02 HS         0.45  
    PER        3.07  

[5 rows x 73 columns]

In [13]:
data.T.head()

0            N01                N02                 N03                 N04  \
1             HS   PER    DIR    HS   PER     DIR    HS   PER     DIR    HS   
0                                                                             
2018022608  0.49  3.14  11.19  0.44  4.03  335.34  0.19  3.68  339.40  0.52   
2018022609  0.48  3.14  11.21  0.43  4.05  335.34  0.19  3.61  341.41  0.51   
2018022610  0.46  3.20  12.50  0.42  4.06  335.40  0.20  3.32  345.22  0.51   
2018022611  0.45  3.27  14.01  0.41  4.04  335.90  0.21  3.05  348.24  0.51   
2018022612  0.43  3.34  15.01  0.40  4.02  336.39  0.24  2.84  351.14  0.52   

0            ...       S48   S49                 S50                 S51  \
1            ...       DIR    HS   PER     DIR    HS   PER     DIR    HS   
0            ...                                                           
2018022608   ...    171.44  0.13  3.10  248.49  0.29  3.93  238.07  0.74   
2018022609   ...    262.09  0.13  3.05  248.99  0.30  3.83  238.61  0.78   
2018022610   ...    259.90  0.14  2.98  249.56  0.32  3.67  239.39  0.83   
2018022611   ...    259.47  0.15  2.90  250.21  0.35  3.45  240.47  0.89   
2018022612   ...    260.42  0.17  2.69  251.63  0.40  3.26  241.54  0.93   

0                         
1            PER     DIR  
0                         
2018022608  4.24  320.09  
2018022609  4.19  320.09  
2018022610  4.18  320.27  
2018022611  4.19  314.53  
2018022612  4.22  306.30  

[5 rows x 639 columns]

#### 存储成csv文件

In [18]:
targetfilename="convert_date.csv"
final_path=os.path.join(targetpath,targetfilename)
print(final_path)

D:\git仓库\GridForecastSys\extra_data\grid_data\convert_date.csv


In [19]:
data.to_csv(final_path)

### 6、读取csv文件

In [30]:
# with open(fullname,"rb") as f:
#     read_date=pd.read_csv(f,',')
# read_date=pd.read_csv(final_path)
read_date=pd.read_csv('convert_date.csv',',')
# read_date=read_date.set_index([0,1])
read_date.head()

,0,1,2018022608,2018022609,2018022610,2018022611,2018022612,2018022613,2018022614,2018022615,...,2018022823,2018030100,2018030101,2018030102,2018030103,2018030104,2018030105,2018030106,2018030107,2018030108
0,N01,HS,0.49,0.48,0.46,0.45,0.43,0.42,0.41,0.40,...,0.35,0.35,0.35,0.35,0.35,0.35,0.36,0.36,0.39,0.42
1,N01,PER,3.14,3.14,3.20,3.27,3.34,3.38,3.41,3.44,...,3.70,3.74,3.81,3.90,3.95,3.93,3.85,3.66,3.30,3.05
2,N01,DIR,11.19,11.21,12.50,14.01,15.01,15.63,16.05,16.31,...,17.03,17.33,17.59,17.84,17.62,16.95,15.89,13.79,9.30,5.26
3,N02,HS,0.44,0.43,0.42,0.41,0.40,0.40,0.39,0.38,...,0.31,0.32,0.32,0.32,0.32,0.32,0.32,0.34,0.39,0.45
4,N02,PER,4.03,4.05,4.06,4.04,4.02,3.99,3.94,3.94,...,4.29,4.36,4.42,4.47,4.44,4.35,4.15,3.76,3.28,3.07


In [36]:
# 注意若使用read_date.set_index([0,1])会提示keyerror的错误
my_date=read_date.set_index(['0','1'])

In [37]:
my_date.head()

2018022608  2018022609  2018022610  2018022611  2018022612  \
0   1                                                                 
N01 HS         0.49        0.48        0.46        0.45        0.43   
    PER        3.14        3.14        3.20        3.27        3.34   
    DIR       11.19       11.21       12.50       14.01       15.01   
N02 HS         0.44        0.43        0.42        0.41        0.40   
    PER        4.03        4.05        4.06        4.04        4.02   

         2018022613  2018022614  2018022615  2018022616  2018022617  \
0   1                                                                 
N01 HS         0.42        0.41        0.40        0.39        0.38   
    PER        3.38        3.41        3.44        3.47        3.49   
    DIR       15.63       16.05       16.31       16.51       16.71   
N02 HS         0.40        0.39        0.38        0.37        0.36   
    PER        3.99        3.94        3.94        3.97        3.98   

            ...      2018022823  2018030100  2018030101  2018030102  \
0   1       ...                                                       
N01 HS      ...            0.35        0.35        0.35        0.35   
    PER     ...            3.70        3.74        3.81        3.90   
    DIR     ...           17.03       17.33       17.59       17.84   
N02 HS      ...            0.31        0.32        0.32        0.32   
    PER     ...            4.29        4.36        4.42        4.47   

         2018030103  2018030104  2018030105  2018030106  2018030107  \
0   1                                                                 
N01 HS         0.35        0.35        0.36        0.36        0.39   
    PER        3.95        3.93        3.85        3.66        3.30   
    DIR       17.62       16.95       15.89       13.79        9.30   
N02 HS         0.32        0.32        0.32        0.34        0.39   
    PER        4.44        4.35        4.15        3.76        3.28   

         2018030108  
0   1                
N01 HS         0.42  
    PER        3.05  
    DIR        5.26  
N02 HS         0.45  
    PER        3.07  

[5 rows x 73 columns]

### 7、获取最大值

In [93]:
data.T.idxmax(axis=1).head()

0
2018022608    (N20, DIR)
2018022609    (N34, DIR)
2018022610    (N04, DIR)
2018022611    (N75, DIR)
2018022612    (N75, DIR)
dtype: object

In [17]:
data_max=data.T.idxmax(axis=0)
data_max.head()

0    1  
N01  HS     2018022716
     PER    2018030103
     DIR    2018022806
N02  HS     2018030108
     PER    2018030102
dtype: object

In [39]:
my_date.T.idxmax(axis=0).head()

0    1  
N01  HS     2018022716
     PER    2018030103
     DIR    2018022806
N02  HS     2018030108
     PER    2018030102
dtype: object